In [8]:
import pandas as pd
import json

import lar_constraints
import lar_generator
from rules_engine import rules_engine
from test_file_generator import test_data_creator
import utils


import logging
import os


import yaml

from lar_constraints import lar_data_constraints

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [9]:
config_file = 'configurations/clean_file_config.yaml'
bank_config = 'configurations/bank1_config.yaml'
geo_config_file='configurations/geographic_data.yaml'
filepaths_file = 'configurations/test_filepaths.yaml'
lar_schema_file="../schemas/lar_schema.json"
ts_schema_file="../schemas/ts_schema.json"

#load config data
print("start initialization of LAR generator")
with open(config_file, 'r') as f:
	# use safe_load instead load
	lar_file_config_data = yaml.safe_load(f)

with open(filepaths_file, 'r') as f:
	filepaths = yaml.safe_load(f)

#load geographic configuration and census data
print("loading geo data")
with open(geo_config_file, 'r') as f:
	geo_config = yaml.safe_load(f)

with open(bank_config, 'r') as f:
	bank_config_data = yaml.safe_load(f)

DEBUG = False

if not os.path.exists(filepaths["log_filepath"]):
	os.makedirs(filepaths["log_filepath"])

logging.basicConfig(filename=filepaths["log_filepath"]+filepaths['log_filename'], format='%(asctime)s %(message)s', 
					datefmt='%m/%d/%Y %I:%M:%S %p', filemode=filepaths['log_mode'], level=logging.INFO)

geographic_data = pd.read_csv(geo_config['geographic_data_file'], delimiter='|', header=0,
	names=geo_config['file_columns'], dtype=object) #instantiate Census file data as dataframe

#create 11 digit Census Tract codes from 5 digit county and 6 digit tract
geographic_data['county_fips'] = geographic_data.apply(lambda x: str(x.state_code) + str(x.county), axis=1)
geographic_data["tract_fips"] = geographic_data.apply(lambda x: str(x.county_fips) + str(x.tracts), axis=1)

with open(geo_config["zip_code_file"], 'r') as f:
	zip_codes = json.load(f)
zip_codes.append("Exempt")

#convert file_generator.py to this script

#instantiate lar generator to create random LAR and fixed TS data
lar_gen = lar_generator.lar_gen(lar_schema_file=lar_schema_file, ts_schema_file=ts_schema_file)

#instantiate rules engine to check conformity of synthetic data to FIG schema
rules_engine = rules_engine(config_data=lar_file_config_data, state_codes=geo_config["state_codes"], state_codes_rev=geo_config["state_codes_rev"],
	geographic_data=geographic_data, full_lar_file_check=False)

#instantiate constraints logic to force LAR data to conform to FIG schema
lar_constraints = lar_data_constraints(lar_file_config=lar_file_config_data, geographic_data=geographic_data)

#store original row for diff comparison to see what elements are being changed

ts_row = lar_gen.make_ts_row(bank_file_config=bank_config_data) #create TS row, we only need one
ts_df = pd.DataFrame(ts_row, index=[0])
rules_engine.load_ts_data(ts_df) #loading ts_row to rules_engine converts it to a dataframe for value checking
lar_rows = [] #list to hold all OrderedDict LAR records before writing to file

test_file_gen = test_data_creator(ts_schema_file=ts_schema_file, lar_schema_file=lar_schema_file, bank_config_data=bank_config_data,
								state_codes=geo_config["state_codes"], county_df=geographic_data[["county_fips", "small_county"]])


start initialization of LAR generator
loading geo data
start initialization of LAR generator
LAR generator initialization complete
initializing rules engine
loading config files for rules engine
opening json schema files
schema loaded
rules engine finished initializing


In [10]:
rules_engine.split_ts_row(data_file="../edits_files/Bank 0/clean_files/Bank 0_clean_1000000_rows.txt")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rules_engine.split_ts_row(data_file="../edits_files/Bank 0/clean_files/Bank 0_clean_1000     │
│   2                                                                                              │
│                                                                                                  │
│ /Users/bringmannj/Documents/hmda-test-files/2022/python/rules_engine.py:96 in split_ts_row       │
│                                                                                                  │
│     93 │   │   Separates TS and LAR portions of a file and returns each as a dataframe.          │
│     94 │   │   """                                                                               │
│     95 │   │                                                                                     │
│ ❱   96 │   │   with open(data_file, 'r') as infile:                                              │
│     97 │   │   │   ts_row = infile.readline().strip("\n")                                        │
│     98 │   │   │   ts_data = []                                                                  │
│     99 │   │   │   ts_data.append(ts_row.split("|"))                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '../edits_files/Bank 0/clean_files/Bank 
0_clean_1000000_rows.txt'

In [5]:
res_df = rules_engine.create_edit_report()
res_df.head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 res_df = rules_engine.create_edit_report()                                                   │
│   2 res_df.head()                                                                                │
│   3                                                                                              │
│                                                                                                  │
│ /Users/bringmannj/Documents/hmda-test-files/2022/python/rules_engine.py:125 in                   │
│ create_edit_report                                                                               │
│                                                                                                  │
│    122 │   │   rules = [rule for rule in rules if rule[:1] in rules_list]                        │
│    123 │   │   for rule in rules:                                                                │
│    124 │   │   │   if rule[:1] in rules_list and rule[1:4].isdigit()==True:                      │
│ ❱  125 │   │   │   │   getattr(self, rule)()                                                     │
│    126 │   │   res_df = pd.DataFrame(self.results)                                               │
│    127 │   │   return res_df                                                                     │
│    128                                                                                           │
│                                                                                                  │
│ /Users/bringmannj/Documents/hmda-test-files/2022/python/rules_engine.py:276 in s300_2            │
│                                                                                                  │
│    273 │   │   """                                                                               │
│    274 │   │   field = "record_id"                                                               │
│    275 │   │   edit_name = "s300_2"                                                              │
│ ❱  276 │   │   fail_df = self.lar_df[self.lar_df.record_id!="2"]                                 │
│    277 │   │   self.results_wrapper(edit_name=edit_name, field_name=field, fail_df=fail_df, row  │
│    278 │                                                                                         │
│    279 │   def s301(self):                                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'rules_engine' object has no attribute 'lar_df'

In [7]:
res_df[res_df.edit_name=="v699"]

,edit_name,row_type,field,fail_count,failed_rows
209,v699,LAR,AUS and Results,0,[]


In [9]:
print(rules_engine.lar_df.columns)

Index(['record_id', 'lei', 'uli', 'app_date', 'loan_type', 'loan_purpose',
       'preapproval', 'const_method', 'occ_type', 'loan_amount',
       ...
       'aus_code_5', 'aus_result_1', 'aus_result_2', 'aus_result_3',
       'aus_result_4', 'aus_result_5', 'aus_code_16', 'reverse_mortgage',
       'open_end_credit', 'business_purpose'],
      dtype='object', length=110)


In [17]:
aus_results = ("1","2","3", "4", "5", "6", "7", "8", "9", "10", 
			"11", "12","13", "14","15", "16")
for index, row in rules_engine.lar_df.iterrows():
    if row["app_sex"] == "6":
        row["app_sex_basis"] = "2"
    if row["co_app_sex"] == "6":
        row["co_app_sex_basis"] = "2"
    if row["aus_1"] == "5":
        if row["aus_result_1"] == "17":
            row["aus_result_1"] == random.choice(aus_results)
    if row["aus_2"] == "5":
        if row["aus_result_2"] == "17":
            row["aus_result_2"] == random.choice(aus_results)
    if row["aus_3"] == "5":
        if row["aus_result_3"] == "17":
            row["aus_result_3"] == random.choice(aus_results)
    if row["aus_4"] == "5":
        if row["aus_result_4"] == "17":
            row["aus_result_4"] == random.choice(aus_results)
    if row["aus_5"] == "5":
        if row["aus_result_5"] == "17":
            row["aus_result_5"] == random.choice(aus_results)
                

In [18]:
utils.write_file(path="../edits_files/Bank 0/clean_files/", 
                 name="new_bank_0_1m_rows.txt", 
                 ts_input=rules_engine.ts_df, lar_input=rules_engine.lar_df)

In [6]:
for col in rules_engine.lar_df.columns:
    print(col)
print(len(rules_engine.lar_df.columns))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 for col in rules_engine.lar_df.columns:                                                      │
│   2 │   print(col)                                                                               │
│   3 print(len(rules_engine.lar_df.columns))                                                      │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'rules_engine' object has no attribute 'lar_df'

In [3]:
print(len(rules_engine.svq_edit_functions))
print(len(test_file_gen.test_file_funcs))

288
297


In [ ]:
clean_file_rep = pd.read_csv("../edit_reports/Bank1_clean_file_report.txt", sep="|")
print(len(clean_file_rep.edit_name), "edits checked")
clean_file_rep.head()

In [ ]:
edit_file_rep = pd.read_csv("../edit_reports/Bank1_edit_file_report.txt", sep="|")
print(len(edit_file_rep.edit_name), "test files created")
edit_file_rep.head()

In [ ]:
clean_edit_list = list(clean_file_rep.edit_name.unique())
fail_edit_list = list(edit_file_rep.edit_name.unique())
rules_eng_list = rules_engine.svq_edit_functions


for rule in rules_eng_list:
    if rule not in fail_edit_list:
        print(rule, "not in report")
    

In [ ]:
for rule in rules_eng_list:
    if rule not in clean_edit_list:
        print(rule, "not in report")

In [ ]:
for rule in test_file_gen.test_file_funcs:
    if rule not in rules_eng_list:
        print(rule, "in test file maker but not rules engine")

In [ ]:
#how many functions in test file creator?

#do these map to rules engine? 
#to edit files


#

In [ ]:
#load 1 million row file
#check for quality edit fails, need 300k
len(test_df)

In [ ]:
rules_engine.split_ts_row(data_file="../edits_files/Bank 0/clean_files/Bank 0_clean_1000000_rows.txt", load=True)

In [ ]:
test_fails = rules_engine.create_edit_report(rules_list=["s", "v", "q"])


In [ ]:
len(test_fails[test_fails.fail_count>0])
test_fails[test_fails.fail_count>0]


In [ ]:
test_fails[test_fails.edit_name.apply(lambda x: x[:1]=="q")]

In [ ]:
test_fails[test_fails.edit_name.apply(lambda x: x[:1]=="m")]

In [ ]:
#load 1m file edit report
rep_df = pd.read_csv("../edit_reports/Bank1_clean_file_report.txt", sep="|")
rep_df.head()

In [ ]:
rep_df[rep_df.fail_count>0]

In [ ]:
"Activity Year|Legal Entity Identifier (LEI)|Agency Code|Institution Type|2017 Institution ID|Tax ID|RSSD|Email Domain|Respondent Name|Respondent State|Respondent City|Parent ID RSSD|Parent Name|Assets|Other Lender Code|Top Holder ID RSSD|Top Holder Name|HMDA Filer Flag".split("|")

In [ ]:
"lei|activityyear|agency|institutiontype|id2017|taxid|rssd|emaildomain|respondentname|respondentstate|respondentcity|parentidrssd|parentname|assets|otherlendercode|topholderidrssd|topholdername|hmdafiler".split("|")